In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

## Librerias

In [2]:
from typing import Union, Tuple, Optional, Callable
import warnings
import logging
import sys
import numpy as np
import pandas as pd
import sklearn
import sklearn.pipeline
from sklearn.base import clone
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from copy import copy, deepcopy
import inspect

import skforecast
from skforecast.ForecasterBase import ForecasterBase
from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.utils import initialize_lags
from skforecast.utils import initialize_weights
from skforecast.utils import check_select_fit_kwargs
from skforecast.utils import check_exog
from skforecast.utils import get_exog_dtypes
from skforecast.utils import check_exog_dtypes
from skforecast.utils import check_interval
from skforecast.utils import check_predict_input
from skforecast.utils import preprocess_y
from skforecast.utils import preprocess_last_window
from skforecast.utils import preprocess_exog
from skforecast.utils import expand_index
from skforecast.utils import transform_series
from skforecast.utils import transform_dataframe
from skforecast.utils import check_preprocess_series
from skforecast.utils import check_preprocess_exog_multiseries

from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from sklearn.linear_model import LinearRegression

## Data

In [3]:
import re
import pytest
import numpy as np
import pandas as pd
from skforecast.exceptions import MissingValuesExogWarning
from skforecast.utils import check_preprocess_series
from skforecast.utils import check_preprocess_exog_multiseries
from skforecast.utils import align_series_and_exog_multiseries

In [4]:
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import series
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import series_2
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import series_as_dict
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog_as_dict
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog_as_dict_datetime
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog_predict

In [5]:
series_diff = {
        'l1': series_as_dict['l1'].copy(),
        'l2': series_as_dict['l2'].copy()
    }
n_nans = 10
series_diff['l1'].iloc[:n_nans, ] = np.nan
series_diff['l1'].iloc[20:23, ] = np.nan
series_diff['l2'].iloc[33:49, ] = np.nan

input_series_is_dict = isinstance(series_diff, dict)
series_dict, series_indexes = check_preprocess_series(series=series_diff)

exog_half_intersection = {
    'l1': exog_as_dict_datetime['l1'].copy(),
    'l2': exog_as_dict_datetime['l2'].copy()
}
exog_half_intersection['l1'].iloc[20:23, :] = np.nan
exog_half_intersection['l1'].index = pd.date_range(
    start='2000-01-15', periods=len(exog_half_intersection['l1']), freq='D'
)
exog_half_intersection['l2'].iloc[:10] = np.nan
exog_half_intersection['l2'].iloc[33:49] = np.nan

In [6]:
exog_half_intersection['l1']

,exog_1,exog_2
2000-01-15,0.513128,a
2000-01-16,0.666625,a
2000-01-17,0.105908,a
2000-01-18,0.130895,a
2000-01-19,0.321981,a
2000-01-20,0.661564,a
2000-01-21,0.846506,a
2000-01-22,0.553257,a
2000-01-23,0.854452,a
2000-01-24,0.384838,a


In [13]:
exog_as_dict_datetime['l1'].head(5)

,exog_1,exog_2
2000-01-01,0.513128,a
2000-01-02,0.666625,a
2000-01-03,0.105908,a
2000-01-04,0.130895,a
2000-01-05,0.321981,a


In [8]:
exog_as_dict_datetime['l2'].head(5)

2000-01-01    0.513128
2000-01-02    0.666625
2000-01-03    0.105908
2000-01-04    0.130895
2000-01-05    0.321981
Freq: D, Name: exog_1, dtype: float64

In [10]:
exog_dict, _ = check_preprocess_exog_multiseries(
                       input_series_is_dict = input_series_is_dict,
                       series_indexes       = series_indexes,
                       series_col_names     = ['l1', 'l2'],
                       exog                 = exog_as_dict_datetime,
                       exog_dict            = {'l1': None, 'l2': None}
                   )

In [11]:
exog_dict['l1'].head(6)

,exog_1,exog_2
2000-01-01,0.513128,a
2000-01-02,0.666625,a
2000-01-03,0.105908,a
2000-01-04,0.130895,a
2000-01-05,0.321981,a
2000-01-06,0.661564,a


In [12]:
exog_dict['l1'].iloc[:5, :] = np.nan
exog_dict['l1'].head(6)

,exog_1,exog_2
2000-01-01,NaN,NaN
2000-01-02,NaN,NaN
2000-01-03,NaN,NaN
2000-01-04,NaN,NaN
2000-01-05,NaN,NaN
2000-01-06,0.661564,a


In [27]:
input_series_is_dict = isinstance(series_as_dict, dict)
series_dict, series_indexes = check_preprocess_series(series=series_as_dict)

exog_dict, _ = check_preprocess_exog_multiseries(
                    input_series_is_dict = input_series_is_dict,
                    series_indexes       = series_indexes,
                    series_col_names     = ['l1', 'l2'],
                    exog                 = exog_as_dict_datetime,
                    exog_dict            = {'l1': None, 'l2': None}
                )

series_dict, exog_dict = align_series_and_exog_multiseries(
                                series_dict          = series_dict,
                                input_series_is_dict = input_series_is_dict,
                                exog_dict            = exog_dict
                            )

expected_series_dict = {
    'l1': pd.Series(
                data  = series_as_dict['l1'].to_numpy(),
                index = pd.date_range(start='2000-01-01', periods=len(series_as_dict['l1']), freq='D'),
                name  = 'l1'
            ),
    'l2': pd.Series(
                data  = series_as_dict['l2'].to_numpy(),
                index = pd.date_range(start='2000-01-01', periods=len(series_as_dict['l2']), freq='D'),
                name  = 'l2'
            ),
}
expected_exog_dict = {
    'l1': pd.DataFrame(
                data    = exog_as_dict_datetime['l1'].to_numpy(),
                index   = pd.date_range(start='2000-01-01', periods=len(exog_as_dict_datetime['l1']), freq='D'),
                columns = ['exog_1', 'exog_2']
            ).astype({'exog_1': float}),
    'l2': pd.DataFrame(
                data    = exog_as_dict_datetime['l2'].to_numpy(),
                index   = pd.date_range(start='2000-01-01', periods=len(exog_as_dict_datetime['l2']), freq='D'),
                columns = ['exog_1']
            ).astype({'exog_1': float}),
}

assert isinstance(series_dict, dict)
assert list(series_dict.keys()) == ['l1', 'l2']
for k in series_dict:
    assert isinstance(series_dict[k], pd.Series)
    pd.testing.assert_series_equal(series_dict[k], expected_series_dict[k])

assert isinstance(exog_dict, dict)
assert list(exog_dict.keys()) == ['l1', 'l2']
for k in exog_dict:
    pd.testing.assert_frame_equal(exog_dict[k], expected_exog_dict[k])
    pd.testing.assert_index_equal(exog_dict[k].index, series_dict[k].index)

In [15]:
series_dict['l1']

2000-01-11    0.343178
2000-01-12    0.729050
2000-01-13    0.438572
2000-01-14    0.059678
2000-01-15    0.398044
2000-01-16    0.737995
2000-01-17    0.182492
2000-01-18    0.175452
2000-01-19    0.531551
2000-01-20    0.531828
2000-01-21         NaN
2000-01-22         NaN
2000-01-23         NaN
2000-01-24    0.611024
2000-01-25    0.722443
2000-01-26    0.322959
2000-01-27    0.361789
2000-01-28    0.228263
2000-01-29    0.293714
2000-01-30    0.630976
2000-01-31    0.092105
2000-02-01    0.433701
2000-02-02    0.430863
2000-02-03    0.493685
2000-02-04    0.425830
2000-02-05    0.312261
2000-02-06    0.426351
2000-02-07    0.893389
2000-02-08    0.944160
2000-02-09    0.501837
2000-02-10    0.623953
2000-02-11    0.115618
2000-02-12    0.317285
2000-02-13    0.414826
2000-02-14    0.866309
2000-02-15    0.250455
2000-02-16    0.483034
2000-02-17    0.985560
2000-02-18    0.519485
2000-02-19    0.612895
Freq: D, Name: l1, dtype: float64

In [23]:
len(exog_dict['l1'])

40

In [18]:
expected_exog_dict = {
    'l1': pd.DataFrame(
                data    = exog_half_intersection['l1'].to_numpy(),
                index   = pd.date_range(start='2000-01-15', periods=len(exog_half_intersection['l1']), freq='D'),
                columns = ['exog_1', 'exog_2']
            ).astype({'exog_2': object}),
    'l2': pd.DataFrame(
                data    = exog_half_intersection['l2'].to_numpy(),
                index   = pd.date_range(start='2000-01-01', periods=len(series_2), freq='D'),
                columns = ['exog_1']
            ),
}

In [19]:
expected_exog_dict['l1']

,exog_1,exog_2
2000-01-15,0.513128,a
2000-01-16,0.666625,a
2000-01-17,0.105908,a
2000-01-18,0.130895,a
2000-01-19,0.321981,a
2000-01-20,0.661564,a
2000-01-21,0.846506,a
2000-01-22,0.553257,a
2000-01-23,0.854452,a
2000-01-24,0.384838,a


In [22]:
expected_exog_dict['l1'].loc['2000-01-11':'2000-01-14', :] = np.nan
expected_exog_dict['l1']

,exog_1,exog_2
2000-01-15,0.513128,a
2000-01-16,0.666625,a
2000-01-17,0.105908,a
2000-01-18,0.130895,a
2000-01-19,0.321981,a
2000-01-20,0.661564,a
2000-01-21,0.846506,a
2000-01-22,0.553257,a
2000-01-23,0.854452,a
2000-01-24,0.384838,a


In [26]:
exog.head()

,exog_1,exog_2
0,0.513128,a
1,0.666625,a
2,0.105908,a
3,0.130895,a
4,0.321981,a


In [31]:
pd.DataFrame(
    data    = np.nan,
    index   = pd.date_range(start='2000-01-11', periods=50-10, freq='D'),
    columns = ['exog_1', 'exog_2']
).head().dtypes

exog_1    float64
exog_2    float64
dtype: object

In [22]:
exog_as_dict = {
    'l1': exog.copy(),
    'l2': exog['exog_1'].copy()
}

exog_as_dict['l1'].head(3)
exog_as_dict['l2'].head(3)

0    0.513128
1    0.666625
2    0.105908
Name: exog_1, dtype: float64

In [23]:
exog_as_dict_datetime = {
    'l1': exog.copy(),
    'l2': exog['exog_1'].copy()
}
exog_as_dict_datetime['l1'].index = pd.date_range(start='2000-01-01', periods=len(exog), freq='D')
exog_as_dict_datetime['l2'].index = pd.date_range(start='2000-01-01', periods=len(exog), freq='D')

exog_as_dict_datetime['l1'].head(3)

,exog_1,exog_2
2000-01-01,0.513128,a
2000-01-02,0.666625,a
2000-01-03,0.105908,a


In [24]:
exog_as_dict['l1'].head(3)

,exog_1,exog_2
0,0.513128,a
1,0.666625,a
2,0.105908,a


In [10]:
exog.to_dict('series')

{'exog_1': 0     0.513128
 1     0.666625
 2     0.105908
 3     0.130895
 4     0.321981
 5     0.661564
 6     0.846506
 7     0.553257
 8     0.854452
 9     0.384838
 10    0.316788
 11    0.354265
 12    0.171082
 13    0.829113
 14    0.338671
 15    0.552370
 16    0.578551
 17    0.521533
 18    0.002688
 19    0.988345
 20    0.905342
 21    0.207636
 22    0.292489
 23    0.520010
 24    0.901911
 25    0.983631
 26    0.257542
 27    0.564359
 28    0.806969
 29    0.394370
 30    0.731073
 31    0.161069
 32    0.600699
 33    0.865864
 34    0.983522
 35    0.079366
 36    0.428347
 37    0.204543
 38    0.450636
 39    0.547764
 40    0.093327
 41    0.296861
 42    0.927584
 43    0.569004
 44    0.457412
 45    0.753526
 46    0.741862
 47    0.048579
 48    0.708697
 49    0.839243
 Name: exog_1, dtype: float64,
 'exog_2': 0     a
 1     a
 2     a
 3     a
 4     a
 5     a
 6     a
 7     a
 8     a
 9     a
 10    a
 11    a
 12    a
 13    a
 14    a
 15    a
 16  

In [15]:
exog.to_dict('series').values()

dict_values([0     0.513128
1     0.666625
2     0.105908
3     0.130895
4     0.321981
5     0.661564
6     0.846506
7     0.553257
8     0.854452
9     0.384838
10    0.316788
11    0.354265
12    0.171082
13    0.829113
14    0.338671
15    0.552370
16    0.578551
17    0.521533
18    0.002688
19    0.988345
20    0.905342
21    0.207636
22    0.292489
23    0.520010
24    0.901911
25    0.983631
26    0.257542
27    0.564359
28    0.806969
29    0.394370
30    0.731073
31    0.161069
32    0.600699
33    0.865864
34    0.983522
35    0.079366
36    0.428347
37    0.204543
38    0.450636
39    0.547764
40    0.093327
41    0.296861
42    0.927584
43    0.569004
44    0.457412
45    0.753526
46    0.741862
47    0.048579
48    0.708697
49    0.839243
Name: exog_1, dtype: float64, 0     a
1     a
2     a
3     a
4     a
5     a
6     a
7     a
8     a
9     a
10    a
11    a
12    a
13    a
14    a
15    a
16    a
17    a
18    a
19    a
20    a
21    a
22    a
23    a
24    a
25    b

In [4]:
print(series.shape)
series.columns = ['l1', 'l2']
series.head(2)

(50, 2)


,l1,l2
0,0.696469,0.120629
1,0.286139,0.826341


In [5]:
# Exog for series l1
exog_l1 = pd.DataFrame({
              'exog1': np.arange(50),
              'exog2': np.arange(50, 100),
          })


# Exog for series l2
exog_l2 = pd.Series({'exog1': np.arange(100, 150)})


# Dictionary with exog for each series
exog = {
    'l1': exog_l1,
    'l2': exog_l2
}

## Create train X y

In [6]:
# Create forecaster
forecaster = ForecasterAutoregMultiSeries(
                 regressor = LinearRegression(),
                 lags = 3,
                 transformer_series = None
             )

In [7]:
# Forecaster attributes needed
max_lag = forecaster.max_lag
lags = forecaster.lags
transformer_series = forecaster.transformer_series
transformer_exog = forecaster.transformer_exog

print(max_lag, lags, transformer_series, transformer_exog)

3 [1 2 3] None None


In [12]:
def _create_lags(
    y: np.ndarray, 
    series_name: str
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Transforms a 1d array into a 2d array (X) and a 1d array (y). Each row
    in X is associated with a value of y and it represents the lags that
    precede it.
    
    Notice that, the returned matrix X_data, contains the lag 1 in the first
    column, the lag 2 in the second column and so on.
    
    Parameters
    ----------
    y : numpy ndarray
        1d numpy ndarray Training time series.
    series_name : str
        Name of the series.

    Returns
    -------
    X_data : numpy ndarray
        2d numpy ndarray with the lagged values (predictors). 
        Shape: (samples - max(self.lags), len(self.lags))
    y_data : numpy ndarray
        1d numpy ndarray with the values of the time series related to each 
        row of `X_data`. 
        Shape: (samples - max(self.lags), )
    
    """
        
    n_splits = len(y) - max_lag
    if n_splits <= 0:
        raise ValueError(
            (f"The maximum lag ({max_lag}) must be less than the length "
                f"of the series '{series_name}', ({len(y)}).")
        )
    
    X_data = np.full(shape=(n_splits, len(lags)), fill_value=np.nan, dtype=float)

    for i, lag in enumerate(lags):
        X_data[:, i] = y[max_lag - lag: -lag]

    y_data = y[max_lag:]
        
    return X_data, y_data


def check_exog_create_train_X_y(series, exog, key=None):
    """
    """
    if len(exog) != len(series):
        if key:
            raise ValueError(
                (f"`exog` must have same number of samples as `series`.\n"
                 f"    length `exog`: {len(exog)} for series {key}\n"
                 f"    length `series`: {len(series)}")
            )
        else:
            raise ValueError(
                (f"`exog` must have same number of samples as `series`. "
                 f"length `exog`: ({len(exog)}), length `series`: ({len(series)})")
            )
    
    check_exog(exog=exog, allow_nan=True)

    if isinstance(exog, pd.Series):
        # Needed for pandas concat
        exog = exog.to_frame()
    
    exog = transform_dataframe(
                df                = exog,
                transformer       = transformer_exog,
                fit               = True,
                inverse_transform = False
            )

    check_exog(exog=exog, allow_nan=False)
    check_exog_dtypes(exog)

    _, _ = preprocess_exog(exog=exog, return_values=False)
    if not (exog.index[:len(series)] == series.index).all():
        if key:
            raise ValueError(
                (f"Different index for `series` and `exog` for series {key}. "
                 f"They must be equal to ensure the correct alignment of values.")
            )
        else:
            raise ValueError(
                ("Different index for `series` and `exog`. They must be equal "
                 "to ensure the correct alignment of values.")
            )
        
    return exog


def create_train_X_y(
    series: pd.DataFrame,
    exog: Optional[Union[pd.Series, pd.DataFrame, dict]]=None
) -> Tuple[pd.DataFrame, pd.Series, pd.Index, pd.Index]:
    """
    Create training matrices from multiple time series and exogenous
    variables.
    
    Parameters
    ----------
    series : pandas DataFrame
        Training time series.
    exog : pandas Series, pandas DataFrame, dict, default `None`
        Exogenous variable/s included as predictor/s. Must have the same
        number of observations as `series` and their indexes must be aligned.

    Returns
    -------
    X_train : pandas DataFrame
        Training values (predictors).
    y_train : pandas Series
        Values (target) of the time series related to each row of `X_train`.
        Shape: (len(series) - self.max_lag, )
    y_index : pandas Index
        Index of `series`.
    y_train_index: pandas Index
        Index of `y_train`.
    
    """

    if not isinstance(series, pd.DataFrame):
        raise TypeError(f"`series` must be a pandas DataFrame. Got {type(series)}.")

    series_col_names = list(series.columns)

    if transformer_series is None:
        transformer_series_ = {serie: None for serie in series_col_names}
    elif not isinstance(transformer_series, dict):
        transformer_series_ = {serie: clone(transformer_series) 
                               for serie in series_col_names}
    else:
        transformer_series_ = {serie: None for serie in series_col_names}
        # Only elements already present in transformer_series_ are updated
        transformer_series_.update(
            (k, v) for k, v in deepcopy(transformer_series).items() 
            if k in transformer_series_
        )
        series_not_in_transformer_series = set(series.columns) - set(transformer_series.keys())
        if series_not_in_transformer_series:
            warnings.warn(
                (f"{series_not_in_transformer_series} not present in `transformer_series`."
                 f" No transformation is applied to these series."),
                 IgnoredArgumentWarning
            )
    
    if exog is not None:

        if isinstance(exog, dict):
            
            exog_ = {serie: None for serie in series_col_names}
            # Only elements already present in exog_ are updated
            exog_.update(
                (k, v) for k, v in deepcopy(exog).items() 
                if k in exog_
            )
            series_not_in_exog = set(series.columns) - set(exog.keys())
            if series_not_in_exog:
                # TODO: review warning
                warnings.warn(
                    (f"{series_not_in_exog} not present in `exog`."
                     f" No transformation is applied to these series."),
                     IgnoredArgumentWarning
                )

            for k, v in exog.items():
                exog[k] = check_exog_create_train_X_y(series=series, exog=v, key=k)
        else:   
            exog = check_exog_create_train_X_y(series=series, exog=exog, key=None)
            
        # TODO: adapt get_exog_dtypes when dict
        # exog_dtypes = get_exog_dtypes(exog=exog)
    
    X_levels = []
    len_series = []
    X_train_col_names = [f"lag_{lag}" for lag in lags]

    for i, serie in enumerate(series.columns):

        y = series[serie]
        y_values = y.to_numpy()

        if np.isnan(y_values).all():
            raise ValueError(f"All values of series '{serie}' are NaN.")
        
        first_no_nan_idx = np.argmax(~np.isnan(y_values))
        y_values = y_values[first_no_nan_idx:]

        if np.isnan(y_values).any():
            raise ValueError(
                (f"'{serie}' Time series has missing values in between or "
                    f"at the end of the time series. When working with series "
                    f"of different lengths, all series must be complete after "
                    f"the first non-null value.")
            )
        
        y = transform_series(
                series            = y.iloc[first_no_nan_idx:],
                transformer       = transformer_series_[serie],
                fit               = True,
                inverse_transform = False
            )

        y_values = y.to_numpy()
        X_train_values, y_train_values = _create_lags(y=y_values, series_name=serie)

        if i == 0:
            X_train = X_train_values
            y_train = y_train_values
        else:
            X_train = np.concatenate((X_train, X_train_values), axis=0)
            y_train = np.concatenate((y_train, y_train_values), axis=0)

        X_level = [serie]*len(X_train_values)
        X_levels.extend(X_level)
        len_series.append(len(y_train_values))
    
    X_levels = pd.Series(X_levels)
    X_levels = pd.get_dummies(X_levels, dtype=float)

    X_train = pd.DataFrame(
                  data    = X_train,
                  columns = X_train_col_names
              )

    if exog is not None:
        # The first `self.max_lag` positions have to be removed from exog
        # since they are not in X_train. Then Exog is cloned as many times 
        # as there are series, taking into account the length of the series.
        if isinstance(exog, dict):
            exog_to_train = []
            for i, key in enumerate(exog):
                exog_to_train.append(exog[key].iloc[-len_series[i]:, ])
        else:
            exog_to_train = [exog.iloc[-length:, ] for length in len_series]

        exog_to_train = pd.concat(exog_to_train).reset_index(drop=True)
    else:
        exog_to_train = None
    
    X_train = pd.concat([X_train, exog_to_train, X_levels], axis=1)
    X_train_col_names = X_train.columns.to_list()

    y_train = pd.Series(
                  data = y_train,
                  name = 'y'
              )

    _, y_index = preprocess_y(y=series, return_values=False)

    y_index_numpy = y_index.to_numpy()
    y_train_index = pd.Index(
                        np.concatenate(
                            [y_index_numpy[-length:, ] 
                             for length in len_series]
                        )
                    )

    return X_train, y_train, y_index, y_train_index

In [13]:
series.head(2)

,l1,l2
0,0.696469,0.120629
1,0.286139,0.826341


In [14]:
exog.keys()

dict_keys(['l1', 'l2'])

In [11]:
len_series = [50, 50]

exog_to_train = []
for i, key in enumerate(exog):

    if isinstance(exog[key], pd.Series):
        exog_ = exog[key].copy().to_frame()
    else:
        exog_ = exog[key]
    
    exog_to_train.append(exog[key].iloc[-len_series[i]:, ])


exog_to_train = pd.concat(exog_to_train).reset_index(drop=True)

In [15]:
# Exog for series l1
exog_l1 = pd.DataFrame({
              'exog1': np.arange(50),
              'exog2': np.arange(50, 100),
          })


# Exog for series l2
exog_l2 = pd.Series(np.arange(100, 150), name='exog1')


# Dictionary with exog for each series
exog = {
    'l1': None,
    'l2': exog_l2
}

In [16]:
# Create forecaster
forecaster = ForecasterAutoregMultiSeries(
                 regressor = LinearRegression(),
                 lags = 3,
                 transformer_series = None
             )

# Forecaster attributes needed
max_lag = forecaster.max_lag
lags = forecaster.lags
transformer_series = forecaster.transformer_series
transformer_exog = StandardScaler()

print(max_lag, lags, transformer_series, transformer_exog)

3 [1 2 3] None StandardScaler()


In [14]:
for k, v in exog.items():
    _ = check_exog_create_train_X_y(series=series, exog=v, key=k)
    print(_.head(2))

      exog1     exog2
0 -1.697749 -1.697749
1 -1.628453 -1.628453
      exog1
0 -1.697749
1 -1.628453


In [17]:
create_train_X_y(series=series, exog=exog)[0]

TypeError: object of type 'NoneType' has no len()

In [16]:
# Create forecaster
forecaster = ForecasterAutoregMultiSeries(
                 regressor = LinearRegression(),
                 lags = 3,
                 transformer_series = None
             )

In [17]:
exog = pd.Series(np.arange(100, 150), name='exog1')

In [18]:
forecaster.create_train_X_y(series=series, exog=exog)[0]

,lag_1,lag_2,lag_3,exog1,l1,l2
0,0.226851,0.286139,0.696469,103,1.0,0.0
1,0.551315,0.226851,0.286139,104,1.0,0.0
2,0.719469,0.551315,0.226851,105,1.0,0.0
3,0.423106,0.719469,0.551315,106,1.0,0.0
4,0.980764,0.423106,0.719469,107,1.0,0.0
...,...,...,...,...,...,...
89,0.593177,0.762548,0.355915,145,0.0,1.0
90,0.691702,0.593177,0.762548,146,0.0,1.0
91,0.151127,0.691702,0.593177,147,0.0,1.0
92,0.398876,0.151127,0.691702,148,0.0,1.0


## Option 2: Parallelitation

In [6]:
# Create forecaster
forecaster = ForecasterAutoregMultiSeries(
                 regressor = LinearRegression(),
                 lags = 3,
                 transformer_series = None
             )

In [19]:
# Forecaster attributes needed
max_lag = forecaster.max_lag
lags = forecaster.lags
exog_col_names = ['exog1', 'exog2']
transformer_series = forecaster.transformer_series
transformer_exog = forecaster.transformer_exog

print(max_lag, lags, exog_col_names, transformer_series, transformer_exog)

3 [1 2 3] ['exog1', 'exog2'] None None


In [76]:
def _create_lags(
    y: np.ndarray, 
    series_name: str
) -> Tuple[np.ndarray, np.ndarray]:
    """    
    """
        
    n_splits = len(y) - max_lag
    if n_splits <= 0:
        raise ValueError(
            (f"The maximum lag ({max_lag}) must be less than the length "
                f"of the series '{series_name}', ({len(y)}).")
        )
    
    X_data = np.full(shape=(n_splits, len(lags)), fill_value=np.nan, dtype=float)

    for i, lag in enumerate(lags):
        X_data[:, i] = y[max_lag - lag: -lag]

    y_data = y[max_lag:]
        
    return X_data, y_data


def create_train_X_y(
    series: pd.DataFrame,
    exog: Optional[Union[pd.Series, pd.DataFrame, dict]]=None
) -> Tuple[pd.DataFrame, pd.Series, pd.Index, pd.Index]:
    """    
    """

    if not isinstance(series, pd.DataFrame):
        raise TypeError(f"`series` must be a pandas DataFrame. Got {type(series)}.")

    series_col_names = list(series.columns)

    if transformer_series is None:
        transformer_series_ = {serie: None for serie in series_col_names}
    elif not isinstance(transformer_series, dict):
        transformer_series_ = {serie: clone(transformer_series) 
                               for serie in series_col_names}
    else:
        transformer_series_ = {serie: None for serie in series_col_names}
        # Only elements already present in transformer_series_ are updated
        transformer_series_.update(
            (k, v) for k, v in deepcopy(transformer_series).items() 
            if k in transformer_series_
        )
        series_not_in_transformer_series = set(series.columns) - set(transformer_series.keys())
        if series_not_in_transformer_series:
            warnings.warn(
                (f"{series_not_in_transformer_series} not present in `transformer_series`."
                    f" No transformation is applied to these series."),
                    IgnoredArgumentWarning
            )

    if exog is not None:
        if not isinstance(exog, dict):
            exog_dict = {serie: exog for serie in series_col_names}
        else:
            exog_dict = {serie: None for serie in series_col_names}
            # Only elements already present in exog_dict are updated
            exog_dict.update(
                (k, v) for k, v in exog.items() 
                if k in exog_dict
            )
            series_not_in_exog = set(series.columns) - set(exog.keys())
            if series_not_in_exog:
                warnings.warn(
                    (f"{series_not_in_exog} not present in `exog`. All values "
                     f"of the exogenous variables for these series will be NaN."),
                     MissingValuesExogWarning
                )
    
    X_levels = []
    exog_levels = []
    X_train_col_names = [f"lag_{lag}" for lag in lags]

    for i, serie in enumerate(series.columns):

        y = series[serie]
        y_index_no_nan = y.dropna().index

        if len(y_index_no_nan) == 0:
            raise ValueError(f"All values of series '{serie}' are NaN.")
        
        first_valid_idx = y.first_valid_index()
        last_valid_idx = y.last_valid_index()
        y_length = len(y.loc[first_valid_idx:last_valid_idx])
        
        y = transform_series(
                series            = y,
                transformer       = transformer_series_[serie],
                fit               = True,
                inverse_transform = False
            )

        y_values = y.to_numpy()
        X_train_values, y_train_values = _create_lags(y=y_values, series_name=serie)

        delete_from_X_train = True
        if delete_from_X_train:
            rows_with_nan = np.isnan(X_train_values).any(axis=1)
        else:
            rows_with_nan = np.isnan(y_train_values)
        
        X_train_values = X_train_values[~rows_with_nan]
        y_train_values = y_train_values[~rows_with_nan]

        if i == 0:
            X_train = X_train_values
            y_train = y_train_values
        else:
            X_train = np.concatenate((X_train, X_train_values), axis=0)
            y_train = np.concatenate((y_train, y_train_values), axis=0)

        X_level = [serie]*len(X_train_values)
        X_levels.extend(X_level)

        print(X_train_values)

        if exog is not None:
            if exog_dict[serie] is not None:

                check_exog(exog=exog_dict[serie], allow_nan=True)

                if len(exog_dict[serie]) != y_length:
                    raise ValueError(
                        (f"`exog` must have same number of samples as series '{serie}'.\n"
                         f"    length `exog`: {len(exog_dict[serie])} for series {serie}\n"
                         f"    length `series`: {y_length}")
                    )
                
                if delete_from_X_train:
                    display(exog_dict[serie].loc[first_valid_idx:last_valid_idx])
                    display(exog_dict[serie].iloc[max_lag:, ])
                    print(rows_with_nan)
                    exog_level = exog_dict[serie].loc[first_valid_idx:last_valid_idx].iloc[max_lag:, ].iloc[~rows_with_nan]
                else:
                    exog_level = exog_dict[serie].iloc[~rows_with_nan]
                
                if not (exog_level.index == y_index_no_nan[max_lag:, ]).all():
                   raise ValueError(
                       ("Different index for `series` and `exog`. They must be equal "
                        "to ensure the correct alignment of values.")
                   )
                
                if isinstance(exog_level, pd.Series):
                    # Needed for pandas concat
                    exog_level = exog_level.to_frame()
            else:
                exog_level = pd.DataFrame(
                                 np.nan, 
                                 index   = np.arange(len(X_train_values)), 
                                 columns = exog_col_names
                             )
            
            exog_levels.append(exog_level)

    X_levels = pd.Series(X_levels)
    X_levels = pd.get_dummies(X_levels, dtype=float)

    X_train = pd.DataFrame(
                  data    = X_train,
                  columns = X_train_col_names
              )

    if exog is not None:
        exog_train = pd.concat(exog_levels).reset_index(drop=True)
        exog_train = transform_dataframe(
                         df                = exog_train,
                         transformer       = transformer_exog,
                         fit               = True,
                         inverse_transform = False
                     )
        
        # check_exog_dtypes llama a check_exog
        check_exog_dtypes(exog_train)
        exog_dtypes = get_exog_dtypes(exog=exog_train)
    else:
        exog_train = None
    
    X_train = pd.concat([X_train, exog_train, X_levels], axis=1)
    X_train_col_names = X_train.columns.to_list()

    y_train = pd.Series(
                  data = y_train,
                  name = 'y'
              )

    _, y_index = preprocess_y(y=series, return_values=False)

    y_train_index = []
    for serie in series.columns:
        serie_index = series[serie].dropna().index[max_lag:, ]
        y_train_index.append(serie_index)

    y_train_index = pd.Index(np.concatenate(y_train_index))

    return X_train, y_train, y_index, y_train_index

In [20]:
series_2 = series.head(10).copy()
series_2.head(2)

,l1,l2
0,0.696469,0.120629
1,0.286139,0.826341


In [21]:
series_3 = series_2.copy()
series_3.iloc[:3, 1] = np.nan
series_3

,l1,l2
0,0.696469,NaN
1,0.286139,NaN
2,0.226851,NaN
3,0.551315,0.545068
4,0.719469,0.342764
5,0.423106,0.304121
6,0.980764,0.417022
7,0.684830,0.681301
8,0.480932,0.875457
9,0.392118,0.510422


In [55]:
# Exog for series l1
exog_l1 = pd.DataFrame({
              'exog1': np.arange(10),
              'exog2': np.arange(50, 60),
          })

# Exog for series l2
exog_l2 = pd.Series(np.arange(100, 107), name='exog1').to_frame()

# Dictionary with exog for each series
exog = {
    'l1': exog_l1,
    'l2': exog_l2
}

In [57]:
# Create forecaster
forecaster = ForecasterAutoregMultiSeries(
                 regressor = LinearRegression(),
                 lags = 3,
                 transformer_series = None
             )

# Forecaster attributes needed
max_lag = forecaster.max_lag
lags = forecaster.lags
transformer_series = forecaster.transformer_series
transformer_exog = forecaster.transformer_exog
# transformer_exog = StandardScaler()

print(max_lag, lags, transformer_series, transformer_exog)

3 [1 2 3] None None


In [58]:
series_create = series_3
series_create

,l1,l2
0,0.696469,NaN
1,0.286139,NaN
2,0.226851,NaN
3,0.551315,0.545068
4,0.719469,0.342764
5,0.423106,0.304121
6,0.980764,0.417022
7,0.684830,0.681301
8,0.480932,0.875457
9,0.392118,0.510422


In [67]:
y = series_3['l2']

first_valid_idx = y.first_valid_index()
last_valid_idx = y.last_valid_index()
y_positions = len(y.loc[first_valid_idx:last_valid_idx])

print(first_valid_idx, last_valid_idx, y_positions) 

3 9 7


In [65]:
series_3['l2'].last_valid_index()

9

In [77]:
create_train_X_y(series=series_create, exog=exog)[0]

[[0.22685145 0.28613933 0.69646919]
 [0.55131477 0.22685145 0.28613933]
 [0.71946897 0.55131477 0.22685145]
 [0.42310646 0.71946897 0.55131477]
 [0.9807642  0.42310646 0.71946897]
 [0.68482974 0.9807642  0.42310646]
 [0.4809319  0.68482974 0.9807642 ]]


,exog1,exog2
0,0,50
1,1,51
2,2,52
3,3,53
4,4,54
5,5,55
6,6,56
7,7,57
8,8,58
9,9,59


,exog1,exog2
3,3,53
4,4,54
5,5,55
6,6,56
7,7,57
8,8,58
9,9,59


[False False False False False False False]
[[0.30412079 0.34276383 0.54506801]
 [0.41702221 0.30412079 0.34276383]
 [0.68130077 0.41702221 0.30412079]
 [0.87545684 0.68130077 0.41702221]]


,exog1
3,103
4,104
5,105
6,106


,exog1
3,103
4,104
5,105
6,106


[ True  True  True False False False False]


IndexError: Boolean index has wrong length: 7 instead of 1

In [138]:
create_train_X_y(series=series_create, exog=exog)[1]

[0.55131477 0.71946897 0.42310646 0.9807642  0.68482974 0.4809319
 0.39211752]
(7,)
[0.54506801 0.34276383 0.30412079 0.41702221 0.68130077 0.87545684
 0.51042234]
(7,)


c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:346: MissingValuesExogWarning: `exog` has missing values. Most machine learning models do not allow missing values. Fitting the forecaster may fail. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesExogWarning)
  warnings.warn(


0     0.551315
1     0.719469
2     0.423106
3     0.980764
4     0.684830
5     0.480932
6     0.392118
7     0.545068
8     0.342764
9     0.304121
10    0.417022
11    0.681301
12    0.875457
13    0.510422
Name: y, dtype: float64

In [121]:
create_train_X_y(series=series_create, exog=exog)[2]

c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:346: MissingValuesExogWarning: `exog` has missing values. Most machine learning models do not allow missing values. Fitting the forecaster may fail. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesExogWarning)
  warnings.warn(


RangeIndex(start=0, stop=10, step=1)

In [119]:
create_train_X_y(series=series_create, exog=exog)[3]

c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:346: MissingValuesExogWarning: `exog` has missing values. Most machine learning models do not allow missing values. Fitting the forecaster may fail. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesExogWarning)
  warnings.warn(


Index([3, 4, 5, 6, 7, 8, 9, 6, 7, 8, 9], dtype='int64')

In [55]:
X_train_rows_with_nan = np.array([])

# exog['l1'].iloc[~X_train_rows_with_nan, ]
exog['l1'][~exog['l1'].index.isin(X_train_rows_with_nan)]

,exog1,exog2
0,0,50
1,1,51
2,2,52
3,3,53
4,4,54
5,5,55
6,6,56
7,7,57
8,8,58
9,9,59


In [9]:
series_3 = pd.DataFrame({
    'l1': np.arange(10),
    'l2': np.arange(50, 60)
})

# series_3.iloc[:5, 1] = np.nan

In [10]:
series_3['l1']

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
Name: l1, dtype: int32

In [102]:
create_train_X_y(series=series_3, exog=exog)[0]

,lag_1,lag_2,lag_3,exog1,exog2,l1,l2
0,2.0,1.0,0.0,-1.5,-1.5,1.0,0.0
1,3.0,2.0,1.0,-1.0,-1.0,1.0,0.0
2,4.0,3.0,2.0,-0.5,-0.5,1.0,0.0
3,5.0,4.0,3.0,0.0,0.0,1.0,0.0
4,6.0,5.0,4.0,0.5,0.5,1.0,0.0
5,7.0,6.0,5.0,1.0,1.0,1.0,0.0
6,8.0,7.0,6.0,1.5,1.5,1.0,0.0
7,52.0,51.0,50.0,-1.5,-1.5,0.0,1.0
8,53.0,52.0,51.0,-1.0,-1.0,0.0,1.0
9,54.0,53.0,52.0,-0.5,-0.5,0.0,1.0


In [58]:
exog

{'l1':    exog1  exog2
 0      0     50
 1      1     51
 2      2     52
 3      3     53
 4      4     54
 5      5     55
 6      6     56
 7      7     57
 8      8     58
 9      9     59,
 'l2':    exog1
 0    100
 1    101
 2    102
 3    103
 4    104
 5    105
 6    106
 7    107
 8    108
 9    109}

In [93]:
serie = pd.Series(np.arange(10000))
display(serie.head(4))
serie.iloc[3] = np.nan
display(serie.head(4))

array = serie.to_numpy()

0    0
1    1
2    2
3    3
dtype: int32

0    0.0
1    1.0
2    2.0
3    NaN
dtype: float64

In [103]:
%%timeit -n 10000 -r 4

# Finding the positions of non-NaN values in the Series
non_nan_positions_mostly_nan = np.argwhere(~np.isnan(array)).flatten()

# Filtering the Series using iloc and the positions of non-NaN values
filtered_series_non_nan_mostly_nan = serie.iloc[non_nan_positions_mostly_nan]
filtered_series_non_nan_mostly_nan


128 µs ± 6.83 µs per loop (mean ± std. dev. of 4 runs, 10,000 loops each)


In [104]:
%%timeit -n 10000 -r 4

serie.dropna()

61.1 µs ± 1.71 µs per loop (mean ± std. dev. of 4 runs, 10,000 loops each)


In [102]:
%%timeit -n 10000 -r 4

serie[~serie.isna()]

129 µs ± 6.53 µs per loop (mean ± std. dev. of 4 runs, 10,000 loops each)


In [8]:
serie = pd.Series([np.nan]*10)
serie.index.dtype.name

'int64'

In [114]:
serie = pd.Series([np.nan]*10)
display(serie.head(4))
serie = serie.dropna().index
len(serie)

0   NaN
1   NaN
2   NaN
3   NaN
dtype: float64

0

In [113]:
display(series.head(4))
serie = series['l1']
serie = serie.dropna().index
display(serie)
display(series.head(4))

,l1,l2
0,0.696469,0.120629
1,0.286139,0.826341
2,0.226851,0.603060
3,NaN,0.545068


Index([ 0,  1,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
      dtype='int64')

,l1,l2
0,0.696469,0.120629
1,0.286139,0.826341
2,0.226851,0.603060
3,NaN,0.545068


In [73]:
l = [[1,2,3,4,5], [1,2,3,4,5]]

pd.Index(np.concatenate(l))

Index([1, 2, 3, 4, 5, 1, 2, 3, 4, 5], dtype='int32')

In [111]:
id(series)

1385573382160

In [140]:
array1_5x6 = np.array([[1, np.nan, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, np.nan, 16, 17, 18], [19, 20, 21, np.nan, 23, 24], [25, 26, 27, 28, 29, 30]])

np.isnan(array1_5x6).any(axis=1)

array([ True, False,  True,  True, False])

In [141]:
array2_1d_5 = np.array([101, 102, 103, 104, 105])
np.isnan(array2_1d_5)

array([False, False, False, False, False])

In [70]:
# Creating a new example where the first array is 5x6 and the second array is 1D with length 5
array1_5x6 = np.array([[1, np.nan, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, np.nan, 16, 17, 18], [19, 20, 21, np.nan, 23, 24], [25, 26, 27, 28, 29, 30]])
array2_1d_5 = np.array([101, 102, 103, 104, 105])

print(array1_5x6)
print(array2_1d_5)

# Find rows in the first array that contain NaN
rows_with_nan = np.unique(np.argwhere(np.isnan(array1_5x6))[:, 0])
# rows_with_nan = np.isnan(array1_5x6).any(axis=1)

print(rows_with_nan)

# Deleting the corresponding rows in the 1D array
# array2_1d_5_filtered = np.delete(array2_1d_5, rows_with_nan)
array2_1d_5_filtered = array2_1d_5[~rows_with_nan]
array2_1d_5_filtered


[[ 1. nan  3.  4.  5.  6.]
 [ 7.  8.  9. 10. 11. 12.]
 [13. 14. nan 16. 17. 18.]
 [19. 20. 21. nan 23. 24.]
 [25. 26. 27. 28. 29. 30.]]
[101 102 103 104 105]
[0 2 3]


array([105, 103, 102])

In [69]:
array = np.array([True, False, True, False, True, False, True, False, True, False])

series_2.iloc[array]

,l1,l2
0,0.696469,0.120629
2,0.226851,0.603060
4,0.719469,0.342764
6,0.980764,0.417022
8,0.480932,0.875457


In [119]:
np.argwhere(np.isnan(array1_5x6))

array([[0, 1],
       [2, 2],
       [3, 3]], dtype=int64)

In [118]:
np.argwhere(np.isnan(array1_5x6))[:, 0]

array([0, 2, 3], dtype=int64)

In [121]:
rows_with_nan

array([0, 2, 3], dtype=int64)

In [131]:
# Creating a 1D array for demonstration
array_1d_example = np.array([10, 20, 30, 40, 50])

# Filtering the 1D array to keep only positions that are not in `rows_with_nan`
filtered_array_1d = array_1d_example[~np.isin(range(len(array_1d_example)), rows_with_nan)]
filtered_array_1d


array([20, 50])

In [122]:
array = np.arange(100000)

In [130]:
%%timeit -n 10000 -r 10

array.shape[0]

93.9 ns ± 25.3 ns per loop (mean ± std. dev. of 10 runs, 10,000 loops each)


In [129]:
%%timeit -n 10000 -r 10

len(array)

38.1 ns ± 0.803 ns per loop (mean ± std. dev. of 10 runs, 10,000 loops each)


In [9]:
pd.DataFrame(np.full(shape=(len([1,2])), fill_value=np.nan, dtype=float))

,0
0,NaN
1,NaN


In [11]:
d = {'l1': pd.DataFrame(np.random.randn(10, 4)),
     'l2': pd.DataFrame(np.random.randn(5, 2))}

df = pd.DataFrame(
            data    = np.random.randn(10, 4),
            columns = ['a', 'b', 'c', 'd']
        )
df.iloc[:3, 1] = np.nan
df

,a,b,c,d
0,0.143142,NaN,0.067125,0.828299
1,0.977843,NaN,-0.308325,-1.424829
2,-0.759145,NaN,-0.469457,-0.217533
3,-0.251364,0.492515,-0.327077,1.537074
4,0.462614,0.229408,-0.269854,-0.261489
5,0.084954,1.272910,-0.171364,0.228384
6,0.622712,-1.194614,-1.386955,-0.776856
7,-0.900504,-1.036776,0.314486,0.011250
8,-0.051812,-0.341742,-1.302850,-0.064076
9,0.108309,1.905705,-1.028453,-0.238287


## Opcion Ximo

In [ ]:
def _create_train_X_y_single_series(
    self,
    y: pd.Series,
    exog: Optional[Union[pd.Series, pd.DataFrame]]=None
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.Series]:
    """
    Create training matrices from univariate time series and exogenous
    variables. This method does not transform the exog variables.
    
    Parameters
    ----------
    y : pandas Series
        Training time series.
    exog : pandas Series, pandas DataFrame, default `None`
        Exogenous variable/s included as predictor/s. Must have the same
        number of observations as `y` and their indexes must be aligned.

    Returns
    -------
    X_train_lags : pandas DataFrame
        Training values of lags
        Shape: (len(y) - self.max_lag, len(self.lags))
    X_train_exog : pandas DataFrame
        Training values of exogenous variables.
        Shape: (len(y) - self.max_lag, len(exog.columns))
    y_train : pandas Series
        Values (target) of the time series related to each row of `X_train`.
        Shape: (len(y) - self.max_lag, )
    
    """

    y_name = y.name
    check_y(y=y)
    y = transform_series(
            series            = y,
            transformer       = self.transformer_series_[y_name],
            fit               = True,
            inverse_transform = False
        )
    
    # TODO: check if this is necessary
    y_values, y_index = preprocess_y(y=y)

    if self.differentiation is not None:
        y_values = self.differentiator_[y_name].fit_transform(y_values)
    
    X_train, y_train = self._create_lags(y=y_values)
    X_train_lags = pd.DataFrame(
                       data    = X_train,
                       columns = self.X_train_col_names,
                       index   = y_index[self.max_lag: ]
                   )

    if self.included_exog:
        if exog is not None:
            # The first `self.max_lag` positions have to be removed from exog
            # since they are not in X_train.
            X_train_exog = exog.iloc[self.max_lag:, ]
        else:
            X_train_exog = pd.DataFrame(
                               data    = np.nan,
                               columns = ['_dummy_exog_col_to_keep_shape'],
                               index   = y_index[self.max_lag: ]
                           )
    else:
        X_train_exog = None

    y_train = pd.Series(
                  data  = y_train,
                  index = y_index[self.max_lag: ],
                  name  = 'y'
              )

    if self.differentiation is not None:
        y_train = y_train.iloc[self.differentiation: ]
        X_train_lags = X_train_lags.iloc[self.differentiation: ]
        if X_train_exog is not None:
            X_train_exog = X_train_exog.iloc[self.differentiation: ]

    # assert X_train_lags.index.equals(y_train.index)
    # assert X_train_exog.index.equals(y_train.index)
                    
    return X_train_lags, X_train_exog, y_train

In [ ]:
def create_train_X_y(
    self,
    series: Union[pd.DataFrame, dict],
    exog: Optional[Union[pd.Series, pd.DataFrame, dict]]=None
) -> Tuple[pd.DataFrame, pd.Series, pd.Index, pd.Index]:
    """
    Create training matrices from multiple time series and exogenous
    variables.
    
    Parameters
    ----------
    series : pandas DataFrame, dict
        Training time series.
    exog : pandas Series, pandas DataFrame, dict, default `None`
        Exogenous variable/s included as predictor/s. Must have the same
        number of observations as `series` and their indexes must be aligned.

    Returns
    -------
    X_train : pandas DataFrame
        Training values (predictors).
    y_train : pandas Series
        Values (target) of the time series related to each row of `X_train`.
        Shape: (len(series) - self.max_lag, )
    y_index : pandas Index
        Index of `series`.
    y_train_index: pandas Index
        Index of `y_train`.
    
    """

    if isinstance(series, pd.DataFrame):
        series_col_names = list(series.columns)
        series_dict = series.to_dict("series")
    elif isinstance(series, dict):
        series_col_names = list(series.keys())
        series_dict = series
    else:
        raise TypeError(f"`series` must be a pandas DataFrame or dict. Got {type(series)}.")
    
    # TODO: incluir reset en fit
    self.series_col_names = series_col_names
    
                
    # Check that all series have the same index type and frequency
    indexes_dtypes = [series.index.dtype.name 
                      for series in series_dict.values()]
    if not len(set(indexes_dtypes)) == 1:
        raise TypeError("All series must have the same index type.")

    indexes_freq = [series.index.freq 
                    if isinstance(series.index, pd.DatetimeIndex) else series.index.step 
                    for series in series_dict.values()]
    if not len(set(indexes_freq)) == 1:
        raise ValueError("All series must have the same frequency/step.")
    
    exog_dict = {serie: None for serie in series_col_names}
    if exog is not None:
        
        if not isinstance(exog, (pd.Series, pd.DataFrame, dict)):
            raise TypeError(
                (f"`exog` must be a pandas Series, DataFrame or dict. "
                 f"Got {type(exog)}.")
            )
        
        if not isinstance(exog, dict):
            exog_dict = {serie: exog for serie in series_col_names}
        else:
            # Only elements already present in exog_dict are updated
            exog_dict.update(
                (k, v) for k, v in exog.items() 
                if k in exog_dict
            )
            series_not_in_exog = set(series_col_names) - set(exog.keys())
            if series_not_in_exog:
                warnings.warn(
                    (f"{series_not_in_exog} not present in `exog`. All values "
                     f"of the exogenous variables for these series will be NaN."),
                     MissingValuesExogWarning
                )
        
        # Convert exog to dataframe if it is a series
        for k, v in exog_dict.items():
            check_exog(exog=v, allow_nan=True)
            if isinstance(v, pd.Series):
                v = v.to_frame()
            exog_dict[k] = v

        exog_col_names = list(set(column for df in exog_dict.values() 
                                  for column in df.columns.to_list()))

        # Check that all exog have the same dtypes for common columns
        exog_dtype_dict = {col_name: set() for col_name in exog_col_names}
        for exog in exog_dict.values():
            for col_name in exog.columns:
                exog_dtype_dict[col_name].add(exog[col_name].dtype.name)
        
        for col_name, dtypes in exog_dtype_dict.items():
            if len(dtypes) > 1:
                raise TypeError(
                    (f"Column {col_name} has different dtypes in different exog "
                     f"DataFrames or Series.")
                )
            
        self.included_exog = True
        self.exog_type = type(exog)
        self.exog_col_names = exog_col_names

        if len(set(self.exog_col_names) - set(self.series_col_names)) != len(self.exog_col_names):
            raise ValueError(
                (f"`exog` cannot contain a column named the same as one of the "
                 f"series (column names of series).\n"
                 f"    `series` columns : {self.series_col_names}.\n"
                 f"    `exog`   columns : {self.exog_col_names}.")
            )

    if self.transformer_series is None:
        self.transformer_series_ = {serie: None for serie in series_col_names}
    elif not isinstance(self.transformer_series, dict):
        self.transformer_series_ = {serie: clone(self.transformer_series) 
                                    for serie in series_col_names}
    else:
        self.transformer_series_ = {serie: None for serie in series_col_names}
        # Only elements already present in transformer_series_ are updated
        self.transformer_series_.update(
            (k, v) for k, v in deepcopy(self.transformer_series).items() 
            if k in self.transformer_series_
        )
        series_not_in_transformer_series = set(series_col_names) - set(self.transformer_series.keys())
        if series_not_in_transformer_series:
            warnings.warn(
                (f"{series_not_in_transformer_series} not present in `transformer_series`."
                    f" No transformation is applied to these series."),
                    IgnoredArgumentWarning
            )
    
    if self.differentiation is None:
        self.differentiator_ = {serie: None for serie in series_col_names}
    else:
        self.differentiator_ = {serie: clone(self.self.differentiator) 
                                for serie in series_col_names}
        
    # Remove leading and trailing nans from each series.
    for k, v in series_dict.items():
        series_dict[k] = v.loc[v.first_valid_index() : v.last_valid_index()]
                
    X_train_lags_buffer = []
    X_train_exog_buffer = []
    y_train_buffer = []

    self.X_train_col_names = [f"lag_{i}" for i in self.lags]
    for key in series_dict.keys():

        y = series_dict[key]
        exog_level = exog_dict[key]

        if self.included_exog:

            if y.index.dtype.name != exog_level.index.dtype.name:
                raise TypeError(
                    (f"Series '{key}' and its `exog` must have the same index type.")
                )
            
            y_freq = y.index.freq if isinstance(y.index, pd.DatetimeIndex) else y.index.step
            exog_freq = exog_level.index.freq if isinstance(exog_level.index, pd.DatetimeIndex) else exog_level.index.step
            if y_freq != exog_freq:
                raise TypeError(
                    (f"Series '{key}' and its `exog` must have the same frequency/step.")
                )
    
            exog_level = exog_level.loc[y.index]
            if len(exog_level) == 0:
                warnings.warn(
                    (f"Series '{key}' and its `exog` do not have the same index. "
                        f"All exog values will be NaN for the period of the series."),
                    MissingValuesExogWarning
                )
                exog_level = exog_level.reindex(y.index, fill_value=np.nan)
            elif len(exog_level) != len(y):
                warnings.warn(
                    (f"Series '{key}' and its `exog` do not have the same length. "
                        f"Exog values will be NaN for the not matched period of the series."),
                    MissingValuesExogWarning
                )
                exog_level = exog_level.reindex(y.index, fill_value=np.nan)

        X_train_lags, X_train_exog, y_train = (
            forecaster._create_train_X_y_single_series(y=y, exog=exog_level)
        )
        X_train_lags['level'] = key
        
        X_train_lags_buffer.append(X_train_lags)
        X_train_exog_buffer.append(X_train_exog)
        y_train_buffer.append(y_train)

    X_train = pd.concat(X_train_lags_buffer, axis=0)
    y_train = pd.concat(y_train_buffer, axis=0)

    if self.included_exog:
        X_train_exog = pd.concat(X_train_exog_buffer, axis=0)
        X_train_exog = X_train_exog.drop(columns=['_dummy_exog_col_to_keep_shape'])
        X_train_exog = transform_dataframe(
                           df                = X_train_exog,
                           transformer       = self.transformer_exog,
                           fit               = True,
                           inverse_transform = False
                       )
        
        check_exog_dtypes(X_train_exog)
        self.exog_dtypes = get_exog_dtypes(exog=X_train_exog)

        # TODO: check if this is necessary
        if not (X_train_exog.index == X_train).all():
            raise ValueError(
                ("Different index for `y` and `exog`. They must be equal "
                 "to ensure the correct alignment of values.")
            )
        X_train = pd.concat([X_train, X_train_exog], axis=1)

    self.X_train_col_names = X_train.columns.to_list()
    y_train_index = y_train.index.to_numpy()


    return X_train, y_train, y_index, y_train_index

In [11]:
df = pd.DataFrame(
        data    = np.random.randn(10, 4),
        columns = ['a', 'b', 'c', 'd']
    )
df


,a,b,c,d
0,-0.434975,-2.052173,-0.459850,0.907756
1,0.338429,1.777685,1.107120,1.257836
2,0.844239,-1.163772,0.763440,-1.015984
3,1.115255,-0.719594,-0.903597,1.326936
4,0.077613,2.126631,-0.325236,1.006936
5,-0.051043,0.321099,0.529483,-0.110667
6,1.643177,-0.629863,0.058391,-0.069865
7,0.144050,2.541392,1.087969,0.366734
8,0.965579,-1.668394,-0.435791,0.511183
9,-1.371704,1.466880,1.102764,0.054621


In [18]:
df.dtypes

a    float64
b    float64
c    float64
d    float64
dtype: object

In [4]:
y = pd.Series(np.arange(7, dtype=float), name='l1')
    
forecaster = ForecasterAutoregMultiSeries(LinearRegression(), lags=3)
forecaster.transformer_series_ = {'l1': StandardScaler()}
results = forecaster._create_train_X_y_single_series(
                y           = y,
                ignore_exog = True,
                exog        = None
            )

In [5]:
results[0]

,lag_1,lag_2,lag_3,_level_skforecast
3,-0.5,-1.0,-1.5,l1
4,0.0,-0.5,-1.0,l1
5,0.5,0.0,-0.5,l1
6,1.0,0.5,0.0,l1


In [6]:
results[1]

In [7]:
isinstance(results[1], type(None))

True

In [30]:
y = pd.Series(np.arange(9, dtype=float), name='l1').to_frame()

transformer = StandardScaler()
transformer = MinMaxScaler()
transformer.fit_transform(y)

array([[0.   ],
       [0.125],
       [0.25 ],
       [0.375],
       [0.5  ],
       [0.625],
       [0.75 ],
       [0.875],
       [1.   ]])

In [31]:
z = pd.Series(np.arange(10, 19, dtype=float), name='l1').to_frame()
transformer.transform(z)

array([[1.25 ],
       [1.375],
       [1.5  ],
       [1.625],
       [1.75 ],
       [1.875],
       [2.   ],
       [2.125],
       [2.25 ]])

In [28]:
z = pd.Series(np.arange(10, 19, dtype=float), name='l1').to_frame()
transformer.fit_transform(z)

array([[0.   ],
       [0.125],
       [0.25 ],
       [0.375],
       [0.5  ],
       [0.625],
       [0.75 ],
       [0.875],
       [1.   ]])

In [3]:
from skforecast.preprocessing import TimeSeriesDifferentiator

In [7]:
y = pd.Series(np.arange(10, dtype=float), name='l1').to_numpy()

transformer = TimeSeriesDifferentiator(order=1)
transformer.fit(y)

TimeSeriesDifferentiator()

In [52]:
y.to_numpy().ndim

1

In [56]:
transformer.transform(y.to_frame().to_numpy())

array([nan])

In [40]:
np.diff(y.to_numpy(), n=1)

array([], shape=(10, 0), dtype=float64)

In [41]:
y = np.array([2, 4.5, 3, 5, 7])

np.diff(y, n=1)

array([ 2.5, -1.5,  2. ,  2. ])

In [42]:
y = np.array([1, 2, 3, 4, 5])
np.diff(y, n=1)

array([1, 1, 1, 1])

In [8]:


series = pd.DataFrame({'1': pd.Series(np.arange(7, dtype=float)), 
                        '2': pd.Series(np.arange(7, dtype=float))})
forecaster = ForecasterAutoregMultiSeries(LinearRegression(), lags=3)

results = forecaster.create_train_X_y(series=series)

In [15]:
results[6]

IndexError: tuple index out of range

In [38]:
def get_exog_dtypes(
    exog: Union[pd.DataFrame, pd.Series]
) -> dict:
    """
    Store dtypes of `exog`.

    Parameters
    ----------
    exog : pandas DataFrame, pandas Series
        Exogenous variable/s included as predictor/s.

    Returns
    -------
    exog_dtypes : dict
        Dictionary with the dtypes in `exog`.
    
    """

    if isinstance(exog, pd.Series):
        exog_dtypes = {exog.name: exog.dtypes}
    else:
        exog_dtypes = {col: exog[col].dtypes.name for col in exog}
        exog_dtypes = exog.dtypes.to_dict()
    
    return exog_dtypes

In [39]:
exog = pd.DataFrame({
        'A': [1, 2, 3], 
        'B': [1.1, 2.2, 3.3],
        'C': pd.Categorical(range(3)),
        'D': [True]*3,
        'E': ['string']*3
    })

# exog = pd.Series([1, 2, 3], name='A')

exog_dtypes = get_exog_dtypes(exog)
exog_dtypes

{'A': dtype('int64'),
 'B': dtype('float64'),
 'C': CategoricalDtype(categories=[0, 1, 2], ordered=False, categories_dtype=int64),
 'D': dtype('bool'),
 'E': dtype('O')}

In [42]:
exog_dtypes['A'].name

'int64'

In [15]:
d = {
    'A': pd.DataFrame({'a': [1, 2, 3]}),
    'B': pd.DataFrame({'b': [1.1, 2.2, 3.3]}),
}

In [16]:
nd = d.copy()

In [17]:
nd['A'].iloc[0, 0] = np.nan

In [18]:
nd

{'A':      a
 0  NaN
 1  2.0
 2  3.0,
 'B':      b
 0  1.1
 1  2.2
 2  3.3}

In [20]:
a = pd.Series([np.nan, 1, 2, 3])

In [22]:
np.isnan(a).all()

False

In [23]:
np.isnan(a.to_numpy()).all()

False

In [43]:
series = pd.DataFrame({'l1': pd.Series(np.arange(10, dtype=float)), 
                           'l2': pd.Series(np.arange(10, dtype=float))})
series.index = np.arange(6, 16)
series

,l1,l2
6,0.0,0.0
7,1.0,1.0
8,2.0,2.0
9,3.0,3.0
10,4.0,4.0
11,5.0,5.0
12,6.0,6.0
13,7.0,7.0
14,8.0,8.0
15,9.0,9.0


In [44]:
exog = pd.Series(np.arange(100, 110), index=np.arange(6, 16), 
                     name='exog', dtype=float)
exog

6     100.0
7     101.0
8     102.0
9     103.0
10    104.0
11    105.0
12    106.0
13    107.0
14    108.0
15    109.0
Name: exog, dtype: float64

In [55]:
series_dict, series_indexes = check_preprocess_series(series)

c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:934: UserWarning: Series has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(


In [56]:
series_dict['l1'].index

RangeIndex(start=0, stop=10, step=1)

In [57]:
series_indexes

{'l1': RangeIndex(start=0, stop=10, step=1),
 'l2': RangeIndex(start=0, stop=10, step=1)}

In [53]:
_, exog_index = preprocess_exog(exog=exog, return_values=False)
exog = exog.copy().to_frame() if isinstance(exog, pd.Series) else exog.copy()
exog.index = exog_index

c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:1059: UserWarning: `exog` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(


In [54]:
exog

,exog
0,100.0
1,101.0
2,102.0
3,103.0
4,104.0
5,105.0
6,106.0
7,107.0
8,108.0
9,109.0


In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [11]:
exog = pd.DataFrame({
               'exog_1': [7.5, 24.4, 60.3, 57.3, 50.7, 41.4, 24.4, 87.2, 47.4, 23.8],
               'exog_2': ['a', 'a', 'a', 'a', 'a', 'b', 'b', 'b', 'b', 'b']},
                index = pd.date_range("1990-01-01", periods=10, freq='D'))

In [12]:
exog_train = exog['exog_1'].iloc[3:].to_frame()
exog_train

,exog_1
1990-01-04,57.3
1990-01-05,50.7
1990-01-06,41.4
1990-01-07,24.4
1990-01-08,87.2
1990-01-09,47.4
1990-01-10,23.8


In [14]:
model = StandardScaler()
model.fit_transform(exog['exog_1'].to_frame())

array([[-1.58551359],
       [-0.81862236],
       [ 0.81045428],
       [ 0.67431975],
       [ 0.37482376],
       [-0.04719331],
       [-0.81862236],
       [ 2.03112731],
       [ 0.22507577],
       [-0.84584926]])

In [13]:
model = StandardScaler()
model.fit_transform(exog_train)

array([[ 0.4908406 ],
       [ 0.16171381],
       [-0.30205575],
       [-1.14980658],
       [ 1.98188469],
       [-0.00284958],
       [-1.17972719]])

In [20]:
series = pd.DataFrame({'l1': np.arange(10, dtype=float), 
                           'l2': pd.Series([np.nan, np.nan, 
                                            2., 3., 4., 5., 6., 7., 8., 9.]), 
                           'l3': pd.Series([np.nan, np.nan, np.nan, np.nan, 
                                            4., 5., 6., 7., 8., 9.])})
series.index = pd.date_range("1990-01-01", periods=10, freq='D')
exog = pd.DataFrame({
            'exog_1': [7.5, 24.4, 60.3, 57.3, 50.7, 41.4, 24.4, 87.2, 47.4, 23.8],
            'exog_2': ['a', 'b', 'a', 'b', 'a', 'b', 'a', 'b', 'a', 'b']},
            index = pd.date_range("1990-01-01", periods=10, freq='D'))

transformer_exog = ColumnTransformer(
                        [('scale', StandardScaler(), ['exog_1']),
                        ('onehot', OneHotEncoder(), ['exog_2'])],
                        remainder = 'passthrough',
                        verbose_feature_names_out = False
                    )

forecaster = ForecasterAutoregMultiSeries(
                    regressor          = LinearRegression(),
                    lags               = 3,
                    transformer_exog   = transformer_exog
                )
results = forecaster.create_train_X_y(series=series, exog=exog)
results[0]

,lag_1,lag_2,lag_3,l1,l2,l3,exog_1,exog_2_a,exog_2_b
1990-01-04,-0.870388,-1.218544,-1.566699,1.0,0.0,0.0,0.426857,0.0,1.0
1990-01-05,-0.522233,-0.870388,-1.218544,1.0,0.0,0.0,0.134812,1.0,0.0
1990-01-06,-0.174078,-0.522233,-0.870388,1.0,0.0,0.0,-0.276705,0.0,1.0
1990-01-07,0.174078,-0.174078,-0.522233,1.0,0.0,0.0,-1.028940,1.0,0.0
1990-01-08,0.522233,0.174078,-0.174078,1.0,0.0,0.0,1.749905,0.0,1.0
1990-01-09,0.870388,0.522233,0.174078,1.0,0.0,0.0,-0.011210,1.0,0.0
1990-01-10,1.218544,0.870388,0.522233,1.0,0.0,0.0,-1.055489,0.0,1.0
1990-01-06,-0.654654,-1.091089,-1.527525,0.0,1.0,0.0,-0.276705,0.0,1.0
1990-01-07,-0.218218,-0.654654,-1.091089,0.0,1.0,0.0,-1.028940,1.0,0.0
1990-01-08,0.218218,-0.218218,-0.654654,0.0,1.0,0.0,1.749905,0.0,1.0


In [19]:
model = StandardScaler()
model.fit_transform(results[0]['exog_1'].to_frame())

array([[ 0.42685655],
       [ 0.13481233],
       [-0.27670452],
       [-1.02893962],
       [ 1.74990535],
       [-0.01120978],
       [-1.0554891 ],
       [-0.27670452],
       [-1.02893962],
       [ 1.74990535],
       [-0.01120978],
       [-1.0554891 ],
       [ 1.74990535],
       [-0.01120978],
       [-1.0554891 ]])

In [6]:
store_l = []

if store_l:
    print(1)



In [12]:
series_col_names = ['l1', 'l2', 'l3']
series_store = ['l1', 'l2']

series_not_in_series_dict = set(series_store) - set(series_col_names)
if series_not_in_series_dict:
    warnings.warn(
        (f"{series_not_in_series_dict} not present in `series`. No 
            last window is stored for them."),
    )